In [8]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
import numpy as np
from data.data_source import get_data_source
from data.treasury_curve import get_yield_curve
from models.pricing_models.bond_model import Bond

ds = get_data_source()

def run_valuation(asof_str):
    asof = pd.to_datetime(asof_str)

    # load inventory
    sql = f"""
    SELECT DISTINCT ON(cusip) cusip, int_rate, issue_date, maturity_date,
        price_per100, quantity, int_payment_frequency
    FROM tsy_inventory
    WHERE inventory_date = '{asof.date()}'
    ORDER BY cusip, inventory_date DESC;
    """
    inv = ds.query(sql).to_pandas()
    if inv.empty:
        print(f"No inventory on {asof.date()}")
        return

    # prepare yield curve
    yc = get_yield_curve(asof, ds)
    # instantiate bonds and gather schedules
    bonds = [Bond(r.cusip, r.issue_date, r.maturity_date,
                  r.int_rate, r.int_payment_frequency,
                  r.price_per100)
             for r in inv.itertuples()]

    # build ragged arrays into matrix
    max_cf = max(len(b.dates) for b in bonds)
    n = len(bonds)
    flows_mat = np.zeros((n, max_cf))
    ttm_mat = np.zeros_like(flows_mat)

    for i, b in enumerate(bonds):
        k = len(b.dates)
        # time to each CF in years
        ttm = (b.dates.astype('datetime64[D]') - asof.to_datetime64()) / np.timedelta64(1, 'D') / 365.25
        flows_mat[i, :k] = b.flows
        ttm_mat[i, :k] = np.where(ttm<0, 0, ttm)

    # vectorized discount factors: exp(-r(t)*t)
    rates = yc(ttm_mat)
    dfs = np.exp(-rates/100 * ttm_mat)
    pvs = (flows_mat * dfs).sum(axis=1)

    # assemble results
    results = inv.copy().reset_index(drop=True)
    results['price_closedform'] = pvs
    results['valuation_date'] = asof.date()
    results['time_to_maturity'] = (pd.to_datetime(results['maturity_date']) - asof).dt.days/365.25
    results['coupon'] = results['int_rate'].fillna(0)

    # batch upsert
    values = []
    for row in results.itertuples(index=False):
        values.append(f"('{row.cusip}','{row.valuation_date}',{row.price_per100},{row.coupon},"
                      f"'{row.maturity_date}',{row.time_to_maturity},{row.price_closedform},{row.quantity})")
    vals_sql = ','.join(values)
    upsert = f"""
    INSERT INTO tsy_valuations (
      cusip, valuation_date, entry_price, coupon, maturity_date,
      time_to_maturity, price_closedform, quantity
    ) VALUES {vals_sql}
    ON CONFLICT(cusip,valuation_date) DO UPDATE SET
      entry_price=EXCLUDED.entry_price,
      coupon=EXCLUDED.coupon,
      maturity_date=EXCLUDED.maturity_date,
      time_to_maturity=EXCLUDED.time_to_maturity,
      price_closedform=EXCLUDED.price_closedform,
      quantity=EXCLUDED.quantity,
      updated_at=CURRENT_TIMESTAMP;
    """
    ds.query(upsert)
    print(f"✅ Valued {n} bonds on {asof.date()}")

if __name__ == '__main__':
    # CLI args in notebook may include flags (-f); only accept YYYY-MM-DD format
    arg = None
    if len(sys.argv) > 1 and not sys.argv[1].startswith('-'):
        arg = sys.argv[1]
    try:
        pd.to_datetime(arg)
        run_valuation(arg)
    except Exception:
        # fallback to today's date if invalid or missing
        # print('hihihih')
        today_str = "2025-05-28"
        print(f"Invalid or missing date '{arg}', defaulting to {today_str}")
        run_valuation(today_str)


getting data source for sandbox
Invalid or missing date 'None', defaulting to 2025-05-28
✅ Valued 355 bonds on 2025-05-28
